In [1]:
import sys,os
homelib = ".."
if not homelib in sys.path:
    sys.path.append(homelib)

from object_publisher import *

In [2]:
class Test:
    
    def __init__(self):
        self.x = "X"
        self.y = "Y"
    
    @publish
    def hello(self):
        return "Hello"
    
    def not_published(self):
        return "private.: %s"%self.x
    
    @publish 
    def world(self, name):
        """
        Worldコマンド
        
        Parameters
        ----------
        name: str
            名前（表示用）
        
        Returns
        -------
        """
        return name
    
    @publish(flask = {"params": "form"})
    def test(self, name, kw=None):
        """
        Testコマンド
        
        Little bit detailed description of Test method.
        
        Parameters
        ----------
        name: str
            名前（表示用）
        kw: str
            表示用のキーワード
        
        Returns
        -------
        """
        return "%s-%s-%s"%(self.y, name, kw)
    
    @publish(flask = {"path": ["id", "attr"], "params": "json"})
    def show_resource(self, id, attr, mustarg, someargs="test"):
        print("TestID: %s"%id)
        print("TestAttr: %s"%attr)
        print("MustArg: %s"%mustarg)
        print("SomeArgs: %s"%someargs)
        
        return {"TestID": id, "TestAttr": attr, "MustArg": mustarg, "SomeArgs": someargs}

In [3]:
obj = Test()
display(obj.hello())
display(obj.not_published())
display(obj.world("me"))
display(obj.show_resource("id1", "attr1", "must"))

'Hello'

'private.: X'

'me'

TestID: id1
TestAttr: attr1
MustArg: must
SomeArgs: test


{'TestID': 'id1', 'TestAttr': 'attr1', 'MustArg': 'must', 'SomeArgs': 'test'}

In [4]:
try:
    CLI(object=obj).run(["-h"])
except SystemExit as e:
    pass

usage: Test [-h] {hello,world,test,show_resource} ...

positional arguments:
  {hello,world,test,show_resource}
                        sub-command help
    hello
    world               Worldコマンド
    test                Testコマンド
    show_resource

optional arguments:
  -h, --help            show this help message and exit


In [5]:
try:
    CLI(klass=Test).run(["hello", "-h"])
except SystemExit as e:
    pass

usage: Test hello [-h]

optional arguments:
  -h, --help  show this help message and exit


In [6]:
CLI(klass=Test).run(["hello"])

'Hello'


In [7]:
try:
    CLI(klass=Test).run(["world", "-h"])
except SystemExit as e:
    pass

usage: Test world [-h] <name>

positional arguments:
  <name>      名前（表示用）

optional arguments:
  -h, --help  show this help message and exit


In [8]:
CLI(object=obj).run(["world", "him"])

'him'


In [9]:
try:
    CLI(object=obj).run(["test", "-h"])
except SystemExit as e:
    pass

usage: Test test [-h] [--kw <kw>] <name>

positional arguments:
  <name>      名前（表示用）

optional arguments:
  -h, --help  show this help message and exit
  --kw <kw>   表示用のキーワード


In [10]:
CLI(object=obj).run(["test", "its"])

'Y-its-None'


In [11]:
CLI(klass=Test).run(["test", "its", "--kw", "TEST"])

'Y-its-TEST'


In [12]:
try:
    CLI(klass=Test).run(["show_resource", "-h"])
except SystemExit as e:
    pass

usage: Test show_resource [-h] [--someargs <someargs>] <id> <attr> <mustarg>

positional arguments:
  <id>
  <attr>
  <mustarg>

optional arguments:
  -h, --help            show this help message and exit
  --someargs <someargs>


In [13]:
CLI(klass=Test).run(["show_resource", "test","ATTR","must!!!","--someargs","keyword"])

TestID: test
TestAttr: ATTR
MustArg: must!!!
SomeArgs: keyword
{'MustArg': 'must!!!',
 'SomeArgs': 'keyword',
 'TestAttr': 'ATTR',
 'TestID': 'test'}


In [14]:
Flask(klass=Test).run(port=9999)

 * Serving Flask app "Test" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2020 00:21:32] "POST /Test/trend/world/show_resource HTTP/1.1" 200 -


TestID: trend
TestAttr: world
MustArg: 1
SomeArgs: test
